# Etiquetamento Autônomo de Perguntas do StackOverflow com Deep Learning

- Bruno Pilão
- Maria Mello
- Larissa Nobrega
- Fernanda Moyses 


O objetivo do trabalho é desenvolver um modelo Deep Learning para o etiquetamento autônomo (*multi-label classification*) de perguntas do StackOverflow.

CONJUNTO DE DADOS

A base de dados a ser utilizada é a StackLite, disponível em https://github.com/dgrtwo/StackLite.

A StackLite é uma versão simplificada e pré-processada de uma parte dos dados do Stack Overflow, contendo perguntas e suas tags associadas.

Embora o repositório possa conter outros arquivos, o foco será nos dados que permitem mapear perguntas (corpo e/ou título) para suas tags.

Obs.: Uma única pergunta pode ter múltiplas tags, o que a torna um problema de classificação multilabel. Pense em uma forma de priorizar uma das tags para simplificar o problema.



In [91]:
# Importando Bibliotecas
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import (
    AutoTokenizer, 
    TFAutoModel,
    TFDistilBertForSequenceClassification
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import ast

## Análise breve dataset 


### Dataset Auxiliar 

Este dataset constitui um dicionário que mapeia cada questão gerada no Stack Overflow às suas respectivas categorias temáticas. O conjunto de dados auxiliar será utilizado para realizar uma análise prévia com o objetivo de avaliar e classificar os tipos de respostas associadas a cada pergunta.

In [92]:
#Importação dataset tags
df_tags = pd.read_csv('data\question_tags.csv\question_tags.csv.gz',compression='gzip')
df_tags.head()

,Id,Tag
0,1,data
1,4,c#
2,4,winforms
3,4,type-conversion
4,4,decimal


In [93]:
#Validandop quantidade de itens na datable
len(df_tags)

52224835

In [94]:
#Validando tipos de tag
print("Quantidade de itens por ID")
for i in range(10):
    print(i+1,df_tags[df_tags['Id'] == i+1]['Tag'].values)

Quantidade de itens por ID
1 ['data']
2 []
3 []
4 ['c#' 'winforms' 'type-conversion' 'decimal' 'opacity']
5 []
6 ['html' 'css' 'css3' 'internet-explorer-7']
7 []
8 ['c#' 'code-generation' 'j#' 'visualj#']
9 ['c#' '.net' 'datetime']
10 []


Observamos que as tags não possuem um ordem sequencial e podem conter um ou multiplos valores dentro de um ID.

In [95]:
# GroupBy simples: ID → lista de tags
grouped_mini = df_tags.groupby('Id')['Tag'].apply(list).reset_index()

print(f"✅ Grouped criado: {len(grouped_mini):,} IDs únicos")

✅ Grouped criado: 17,763,486 IDs únicos


In [ ]:

# Salvar em CSV ()
grouped_mini.to_csv('stackoverflow_grouped_tags.csv', index=False)
print("✅ Backup CSV: stackoverflow_grouped_tags.csv")

✅ Grouped criado: 17,763,486 IDs únicos
✅ Backup CSV: stackoverflow_grouped_tags.csv


In [69]:
# Comprimir CSV (reduz ~80%)
df = pd.read_csv('stackoverflow_grouped_tags.csv')
df.to_csv('stackoverflow_grouped_tags.csv.gz', compression='gzip', index=False)

Com isso avaliamos que este dataset serve como um "Dicionário" para identificar o tipo de pergunta que o usuario realizou no stackoverflow, sendo que uma tg pode contem um ou múltiplos assuntos envolvidos.


### Dataset Principal

O dataset principal consiste em perguntas técnicas geradas por usuários da plataforma Stack Overflow. Este conjunto de dados contém questões formuladas por desenvolvedores e profissionais de tecnologia que buscam soluções para problemas específicos em suas áreas de atuação.

In [96]:
#Importação dataset questions
df_questions = pd.read_csv('data\question_tags.csv\questions.csv.gz',compression='gzip')
df_questions.head()

,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount
0,1,2008-07-31T21:26:37Z,NaN,2011-03-28T00:53:47Z,1,NaN,0.0
1,4,2008-07-31T21:42:52Z,NaN,NaN,472,8.0,13.0
2,6,2008-07-31T22:08:08Z,NaN,NaN,210,9.0,5.0
3,8,2008-07-31T23:33:19Z,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0
4,9,2008-07-31T23:40:59Z,NaN,NaN,1452,1.0,58.0


In [97]:
# Quantiade de itens df_qustions

len(df_questions)

17763486

In [ ]:
# Validar tipos de variáveis ao nosso dataset
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17763486 entries, 0 to 17763485
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Id            int64  
 1   CreationDate  object 
 2   ClosedDate    object 
 3   DeletionDate  object 
 4   Score         int64  
 5   OwnerUserId   float64
 6   AnswerCount   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 948.7+ MB


## Merge Datasets

Vamos agrupar nosso dataset para melhor perfomace no treinamento do nosso modelo para predição de tag do stack overflow

In [99]:
# Vamos realizar um merge de nosso dataset 
df = df_questions.merge(grouped_mini,on='Id',how='inner')

# Verificar resultado
print(f"Dataset final: {df.shape}")
df.head()

Dataset final: (17763486, 8)


,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount,Tag
0,1,2008-07-31T21:26:37Z,NaN,2011-03-28T00:53:47Z,1,NaN,0.0,[data]
1,4,2008-07-31T21:42:52Z,NaN,NaN,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]"
2,6,2008-07-31T22:08:08Z,NaN,NaN,210,9.0,5.0,"[html, css, css3, internet-explorer-7]"
3,8,2008-07-31T23:33:19Z,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0,"[c#, code-generation, j#, visualj#]"
4,9,2008-07-31T23:40:59Z,NaN,NaN,1452,1.0,58.0,"[c#, .net, datetime]"


In [100]:
# Para uma limpeza visual no dataset vamos remover a coluna ID 
df = df.drop("Id",axis=1)

# Verificar resultado
print(f"Dataset final: {df.shape}")
df.head()

Dataset final: (17763486, 7)


,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount,Tag
0,2008-07-31T21:26:37Z,NaN,2011-03-28T00:53:47Z,1,NaN,0.0,[data]
1,2008-07-31T21:42:52Z,NaN,NaN,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]"
2,2008-07-31T22:08:08Z,NaN,NaN,210,9.0,5.0,"[html, css, css3, internet-explorer-7]"
3,2008-07-31T23:33:19Z,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0,"[c#, code-generation, j#, visualj#]"
4,2008-07-31T23:40:59Z,NaN,NaN,1452,1.0,58.0,"[c#, .net, datetime]"


Com isso vamos utilizar este dataframe para realizar uma predição com uma frase de texto bruta e acimiliar o tipo de conteudo que estou relatando

## Treinamento de Modelo

Vamos inicialmente ajustar o nosso treinamento do modelo para que com uma frase de entrada o nosso modelo consiga captar 

In [101]:
# Separar coluna alvo para as de treino 

features = ["CreationDate",	"ClosedDate","DeletionDate"	,"Score"	,"OwnerUserId"	,"AnswerCount"]

X = df[features]
y=df['Tag']

# Divide os dados em conjunto de treinamento e teste (70% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=42,
)

print(f'Base de Treinamento: {X_train.shape}')
print(f'Base de Teste: {X_test.shape}')


Base de Treinamento: (3552697, 6)
Base de Teste: (14210789, 6)
